<a href="https://colab.research.google.com/github/gin7018/image-classifier-convo-nn/blob/main/image_classifier_convolutional_nn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

In [29]:
import torch
import torchvision
import transformers

class ImageClassifierModel(transformers.PreTrainedModel):

  def __init__(self, num_labels=10):
    model_config = transformers.PretrainedConfig(
        num_channels=[32, 64],
        kernel_sizes=[3, 3],
        hidden_size=1600,
        num_classes=num_labels,
        pool_type="max"

    )
    super(ImageClassifierModel, self).__init__(config=model_config)

    self.convo1= torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
    self.convo2= torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
    self.max_pooling1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.convo3= torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.convo4= torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
    self.max_pooling2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

    self.connected_layer1 = torch.nn.Linear(in_features=1600, out_features=128)
    self.activation_fun = torch.nn.ReLU()
    self.connected_layer2 = torch.nn.Linear(in_features=128, out_features=num_labels)

  def forward(self, image_input):
    output = self.convo1(image_input)
    output= self.convo2(output)
    output = self.max_pooling1(output)

    output = self.convo3(output)
    output = self.convo4(output)
    output = self.max_pooling2(output)

    output = output.reshape(output.size(0), -1)

    output = self.connected_layer1(output)
    output = self.activation_fun(output)
    output = self.connected_layer2(output)
    return output

In [3]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

data_transformer = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])

training_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    transform=data_transformer,
    download=True
)

validation_set = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    transform=data_transformer,
    download=True
)

100%|██████████| 170498071/170498071 [00:05<00:00, 28753153.36it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [30]:
# model training parameters
BATCH_SIZE = 64
EPOCHS = 21
LEARNING_RATE = 0.001 # TOO LOW OR TOO HIGH of a lr leads to convergence issues (nan) during training

training_data_loader = DataLoader(
    dataset=training_set,
    batch_size=BATCH_SIZE,
    sampler=RandomSampler(training_set)
)

validation_data_loader = DataLoader(
    dataset=validation_set,
    batch_size=BATCH_SIZE,
    sampler=SequentialSampler(validation_set)
)

In [32]:
from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = ImageClassifierModel()
model.to(device)

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=LEARNING_RATE
)
loss_function = torch.nn.CrossEntropyLoss()

best_accuracy = 0.0
for epoch in range(EPOCHS):
  model.train()
  training_progress_bar = tqdm(training_data_loader, desc=f"Epoch {epoch + 1} - Training")
  for idx, (images, target_labels) in enumerate(training_progress_bar):
    images = images.to(device)
    target_labels = target_labels.to(device)

    outputs = model(images)
    loss = loss_function(outputs, target_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    training_progress_bar.set_postfix(loss=loss.item())


  # testing how good our model is at classifying the sentences
  model.eval()
  total_correct = 0
  total_samples = 0
  validation_progress_bar = tqdm(validation_data_loader, desc=f"Epoch {epoch + 1} - Testing")
  with torch.no_grad():
    for idx, (images, target_labels) in enumerate(validation_progress_bar):
      images = images.to(device)
      target_labels = target_labels.to(device)

      outputs = model(images)
      _, predictions = torch.max(outputs.data, 1)
      total_correct += (predictions == target_labels).sum().item()
      total_samples += target_labels.size(0)
    print(f"epoch: {epoch+1}, accuracy: {100 * (total_correct / total_samples)}")

  current_accuracy = total_correct / total_samples
  if current_accuracy > best_accuracy:
    best_accuracy = current_accuracy
    print("saving to huggingface...")
    model.save_pretrained(
      save_directory="./model",
      state_dict=model.state_dict(),
      push_to_hub=True,
      repo_id="ghislainehaha/image-classifier-cnn",
    )




Epoch 1 - Testing: 100%|██████████| 157/157 [00:03<00:00, 52.02it/s]


epoch: 1, accuracy: 63.4
saving to huggingface...


model.safetensors:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

Epoch 2 - Testing: 100%|██████████| 157/157 [00:02<00:00, 58.52it/s]


epoch: 2, accuracy: 66.02
saving to huggingface...


...

epoch: 19, accuracy: 67.14


Epoch 20 - Testing: 100%|██████████| 157/157 [00:03<00:00, 45.93it/s]


epoch: 20, accuracy: 67.80000000000001


Epoch 21 - Testing: 100%|██████████| 157/157 [00:02<00:00, 58.91it/s]

epoch: 21, accuracy: 67.66
